# FEM Laplace Potential Flow: Inlet Condition Animations

This notebook builds a mesh, solves Laplace potential flow, and visualizes three time-varying inlet conditions.

In [1]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path('..').resolve()
SRC_DIR = PROJECT_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

os.environ.setdefault('MPLBACKEND', 'Agg')

import numpy as np
import matplotlib
matplotlib.use('Agg', force=True)
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

from meshgen.domain import (
    airfoil_offset_rings,
    build_point_cloud,
    gen_outer_boundary,
    make_offset_distances,
    sample_farfield_points,
)
from meshgen.naca4 import gen_naca4
from meshgen.triangulate import generate_mesh

from fem.laplace.solve import MeshGeometry, solve_laplace
from fem.laplace.postprocess import plot_scalar_field


In [ ]:
import time, importlib, sys
mods = [
    "matplotlib",
    "matplotlib.pyplot",
    "matplotlib.animation",
    "IPython.display",
    "meshgen.domain",
    "meshgen.naca4",
    "meshgen.triangulate",
    "fem.laplace.solve",
    "fem.laplace.postprocess",
]
for m in mods:
    t0 = time.time()
    print(f"importing {m}...", flush=True)
    importlib.import_module(m)
    print(f"{m} ok in {time.time()-t0:.2f}s", flush=True)


In [2]:
rng = np.random.default_rng(0)

code = "2412"
chord = 1.0
n_chord = 120
outer_radius_factor = 8.0
n_outer = 200
n_far = 1200
base_frac = 0.006
n_layers = 4
growth = 2.0

airfoil = gen_naca4(code, chord=chord, n_chord=n_chord)
outer = gen_outer_boundary(radius=outer_radius_factor * chord, n_points=n_outer, center=(0.5 * chord, 0.0))

distances = make_offset_distances(base=base_frac * chord, n_layers=n_layers, growth=growth)
near = airfoil_offset_rings(airfoil, distances=distances)

far = sample_farfield_points(
    outer_boundary=outer,
    airfoil_boundary=airfoil,
    n_points=n_far,
    min_dist_to_airfoil=distances[-1],
    rng=rng,
)

points = build_point_cloud(
    airfoil_boundary=airfoil,
    outer_boundary=outer,
    nearfield_points=near,
    farfield_points=far,
    deduplicate=True,
    dedup_tol=1e-12,
)

points, triangles = generate_mesh(points, outer_boundary=outer, airfoil_boundary=airfoil)
geom = MeshGeometry(points=points, triangles=triangles, airfoil_boundary=airfoil, outer_boundary=outer)


In [3]:
# Base solutions for unit x- and y-aligned farfield potentials.
# Linearity lets us superimpose these for time-varying inlets.
phi_x = solve_laplace(lambda x, y: x, geom)
phi_y = solve_laplace(lambda x, y: y, geom)


In [4]:
fig, ax = plt.subplots(figsize=(6, 5))
plot_scalar_field(points, triangles, phi_x, ax=ax, levels=50, cmap="viridis")
ax.set_title("Baseline: phi for farfield = x")
plt.show()


C:\Users\aniru\AppData\Local\Temp\ipykernel_217600\3631581572.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [5]:
def _remove_contour(contour):
    if hasattr(contour, 'collections'):
        for coll in contour.collections:
            coll.remove()
    else:
        contour.remove()


def animate_condition(name, ux_func, uy_func, n_frames=40, interval=120):
    times = np.linspace(0.0, 1.0, n_frames)
    ux_vals = ux_func(times)
    uy_vals = uy_func(times)

    phi_min = np.inf
    phi_max = -np.inf
    for ux, uy in zip(ux_vals, uy_vals):
        phi = ux * phi_x + uy * phi_y
        phi_min = min(phi_min, float(phi.min()))
        phi_max = max(phi_max, float(phi.max()))

    levels = np.linspace(phi_min, phi_max, 50)

    fig, ax = plt.subplots(figsize=(6, 5))
    phi0 = ux_vals[0] * phi_x + uy_vals[0] * phi_y
    contour = ax.tricontourf(points[:, 0], points[:, 1], triangles, phi0, levels=levels, cmap='viridis')
    cbar = fig.colorbar(contour, ax=ax, label='phi')
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    def update(frame):
        nonlocal contour
        _remove_contour(contour)
        phi = ux_vals[frame] * phi_x + uy_vals[frame] * phi_y
        contour = ax.tricontourf(points[:, 0], points[:, 1], triangles, phi, levels=levels, cmap='viridis')
        cbar.update_normal(contour)
        ax.set_title(f"{name} | t={times[frame]:.2f}")
        return []

    anim = animation.FuncAnimation(fig, update, frames=n_frames, interval=interval, blit=False)
    plt.close(fig)
    return anim


In [6]:
# Condition 1: oscillating freestream magnitude along x.
ux1 = lambda t: 1.0 + 0.5 * np.sin(2.0 * np.pi * t)
uy1 = lambda t: 0.0 * t

anim1 = animate_condition("Condition 1: Oscillating Ux", ux1, uy1)
HTML(anim1.to_jshtml())


AttributeError: 'TriContourSet' object has no attribute 'collections'

In [7]:
# Condition 2: ramped angle of attack at 10 degrees.
theta = np.deg2rad(10.0)
ux2 = lambda t: (0.2 + 0.8 * t) * np.cos(theta)
uy2 = lambda t: (0.2 + 0.8 * t) * np.sin(theta)

anim2 = animate_condition("Condition 2: Ramped AoA 10 deg", ux2, uy2)
HTML(anim2.to_jshtml())


AttributeError: 'TriContourSet' object has no attribute 'collections'

In [8]:
# Condition 3: oscillating angle of attack around 0 deg.
theta_osc = lambda t: np.deg2rad(12.0) * np.sin(2.0 * np.pi * t)
ux3 = lambda t: np.cos(theta_osc(t))
uy3 = lambda t: np.sin(theta_osc(t))

anim3 = animate_condition("Condition 3: Oscillating AoA", ux3, uy3)
HTML(anim3.to_jshtml())


AttributeError: 'TriContourSet' object has no attribute 'collections'